In [44]:
import os
import pandas as pd 

In [45]:
data_dir = 'data'
file_list = os.listdir(data_dir)
file_list

['2023년도 데이터.xlsx',
 '2023년.csv',
 '2022년도 데이터.xlsx',
 '2022년.csv',
 '2021년도 데이터.xlsx',
 '2021년.csv',
 'total_df.pkl',
 'total_df.csv']

In [46]:
ob_list = [file for file in file_list if file.endswith('xlsx')]
ob_list

['2023년도 데이터.xlsx', '2022년도 데이터.xlsx', '2021년도 데이터.xlsx']

In [47]:
we_list = [file for file in file_list if file.endswith('csv')]
we_list

['2023년.csv', '2022년.csv', '2021년.csv', 'total_df.csv']

### 측정소 파일 통합

In [48]:
%%time
ob_df_list = []
for file in ob_list:
    df = pd.read_excel(os.path.join(data_dir, file))
    ob_df_list.append(df)

CPU times: total: 1min 15s
Wall time: 1min 15s


In [49]:
ob_df = pd.concat(ob_df_list)
ob_df.reset_index(drop = True, inplace = True)

In [50]:
ob_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595516 entries, 0 to 595515
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   측정소코드   595516 non-null  int64  
 1   측정소명    595516 non-null  object 
 2   일시      595516 non-null  object 
 3   PM2.5   583563 non-null  float64
 4   풍향      594970 non-null  float64
 5   풍속      594860 non-null  float64
 6   온도      594371 non-null  float64
 7   습도      594368 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 36.3+ MB


In [51]:
ob_df.head()

,측정소코드,측정소명,일시,PM2.5,풍향,풍속,온도,습도
0,111261,강남구,2023/01/01 00:00:00,37.0,99.0,1.0,1.6,62.0
1,111261,강남구,2023/01/01 01:00:00,53.0,276.0,0.8,1.1,66.0
2,111261,강남구,2023/01/01 02:00:00,53.0,322.0,1.1,2.3,63.0
3,111261,강남구,2023/01/01 03:00:00,56.0,281.0,1.3,2.4,65.0
4,111261,강남구,2023/01/01 04:00:00,59.0,299.0,1.8,1.9,63.0


In [52]:
ob_df.일시 = ob_df.일시.map(lambda st: st.replace('/','-'))
ob_df.head()

,측정소코드,측정소명,일시,PM2.5,풍향,풍속,온도,습도
0,111261,강남구,2023-01-01 00:00:00,37.0,99.0,1.0,1.6,62.0
1,111261,강남구,2023-01-01 01:00:00,53.0,276.0,0.8,1.1,66.0
2,111261,강남구,2023-01-01 02:00:00,53.0,322.0,1.1,2.3,63.0
3,111261,강남구,2023-01-01 03:00:00,56.0,281.0,1.3,2.4,65.0
4,111261,강남구,2023-01-01 04:00:00,59.0,299.0,1.8,1.9,63.0


### 측정소별 결측값 제거

In [53]:
ob_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595516 entries, 0 to 595515
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   측정소코드   595516 non-null  int64  
 1   측정소명    595516 non-null  object 
 2   일시      595516 non-null  object 
 3   PM2.5   583563 non-null  float64
 4   풍향      594970 non-null  float64
 5   풍속      594860 non-null  float64
 6   온도      594371 non-null  float64
 7   습도      594368 non-null  float64
dtypes: float64(5), int64(1), object(2)
memory usage: 36.3+ MB


In [54]:
ob_df.rename(columns={'PM2.5':'PM'}, inplace = True)

In [67]:
ob_df = ob_df.loc[:, ['측정소명', '일시', 'PM']]
ob_df

,측정소명,일시,PM
0,강남구,2023-01-01 00:00:00,37.0
1,강남구,2023-01-01 01:00:00,53.0
2,강남구,2023-01-01 02:00:00,53.0
3,강남구,2023-01-01 03:00:00,56.0
4,강남구,2023-01-01 04:00:00,59.0
...,...,...,...
595511,중랑구,2021-12-31 19:00:00,7.0
595512,중랑구,2021-12-31 20:00:00,8.0
595513,중랑구,2021-12-31 21:00:00,7.0
595514,중랑구,2021-12-31 22:00:00,7.0


In [77]:
null_dt =  ob_df[ob_df.PM.isnull()]['일시']
null_dt

615       2023-01-26 16:00:00
947       2023-02-09 13:00:00
1089      2023-02-15 11:00:00
2146      2023-03-31 13:00:00
2190      2023-04-02 09:00:00
                 ...         
595068    2021-12-13 04:00:00
595069    2021-12-13 05:00:00
595070    2021-12-13 06:00:00
595071    2021-12-13 07:00:00
595072    2021-12-13 08:00:00
Name: 일시, Length: 11953, dtype: object

In [70]:
total_df = ob_df[~ob_df.일시.isin(null_dt)]
total_df

,측정소명,일시,PM
0,강남구,2023-01-01 00:00:00,37.0
1,강남구,2023-01-01 01:00:00,53.0
2,강남구,2023-01-01 02:00:00,53.0
3,강남구,2023-01-01 03:00:00,56.0
4,강남구,2023-01-01 04:00:00,59.0
...,...,...,...
595507,중랑구,2021-12-31 15:00:00,5.0
595511,중랑구,2021-12-31 19:00:00,7.0
595512,중랑구,2021-12-31 20:00:00,8.0
595514,중랑구,2021-12-31 22:00:00,7.0


In [71]:
total_df.to_pickle('./data/total_df.pkl')

In [ ]:
total_df[(total_df.측정소명 == '강남구') & (total_df.일시 > '2023-09-29')]

In [80]:
ob_df.일시.isin(null_dt)

0         False
1         False
2         False
3         False
4         False
          ...  
595511    False
595512    False
595513     True
595514    False
595515    False
Name: 일시, Length: 595516, dtype: bool